In [11]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dot, LSTM, Dense, Concatenate, Activation
from tensorflow.keras.callbacks import EarlyStopping

cal_len = 5 #10으로 하면 (Seq2Seq 방식) 정확도가 떨어짐 LSTM 이라
chars = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
num_classes = len(chars)

char_to_index = {char: idx for idx, char in enumerate(chars)}
index_to_char = {idx: char for idx, char in enumerate(chars)}

def generate_data(num_samples):
    x = []
    y = []
    for _ in range(num_samples):
        sequence = np.random.choice(list(chars), 15)
        x.append([char_to_index[char] for char in sequence])
        y.append([char_to_index[char] for char in sequence[::-1]])
    return np.array(x), np.array(y)

num_samples = 20000
x, y = generate_data(num_samples)

        
x = x.reshape(x.shape[0], x.shape[1], 1)
y = y.reshape(y.shape[0], y.shape[1], 1)

#encoder
encoder_inputs = Input(shape=(15, 1))
encoder = LSTM(128, return_sequences = True, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

#decoder
decoder_inputs = Input(shape=(15, 1))
decoder = LSTM(128, return_state=True, return_sequences = True)
decoder_outputs, _, _ = decoder(decoder_inputs, initial_state = encoder_states)

#attention mechanism
attention = Dot(axes=[2, 2])([decoder_outputs, encoder_outputs])
attention = Activation('softmax')(attention)
context = Dot(axes=[2, 1])([attention, encoder_outputs])

decoder_combined_context = Concatenate(axis=-1)([decoder_outputs, context])

decoder_dense = Dense(num_classes, activation='softmax')
decoder_outputs = decoder_dense(decoder_combined_context)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

decoder_input_data = np.zeros_like(x)

early_stop = EarlyStopping(monitor='val_loss', patience = 5, restore_best_weights = True)

history= model.fit([x, decoder_input_data], y, epochs=500, batch_size=32, validation_split=(0.2), callbacks=[early_stop])

def predict_reverse(input_sequence):
    input_sequence = np.array([char_to_index[char] for char in input_sequence]).reshape((1, 15, 1))
    decoder_input = np.zeros((1, 15, 1))
    predicted_sequence = model.predict([input_sequence, decoder_input])
    predicted_indices = np.argmax(predicted_sequence, axis=-1).reshape((15, ))
    return ''.join([index_to_char[idx] for idx in predicted_indices])

test_strings = ["".join(np.random.choice(list(chars), 15)) for _ in range(30)]

expected_outputs = [s[::-1] for s in test_strings]

Epoch 1/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 16s 27ms/step - accuracy: 0.0391 - loss: 3.7906 - val_accuracy: 0.2002 - val_loss: 2.6035
Epoch 2/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 12s 25ms/step - accuracy: 0.2554 - loss: 2.4249 - val_accuracy: 0.3752 - val_loss: 2.0076
Epoch 3/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 12s 23ms/step - accuracy: 0.4312 - loss: 1.9072 - val_accuracy: 0.5334 - val_loss: 1.6664
Epoch 4/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - accuracy: 0.5586 - loss: 1.6051 - val_accuracy: 0.6339 - val_loss: 1.4353
Epoch 5/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - accuracy: 0.6466 - loss: 1.3874 - val_accuracy: 0.6797 - val_loss: 1.2621
Epoch 6/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - accuracy: 0.7111 - loss: 1.2192 - val_accuracy: 0.7619 - val_loss: 1.1188
Epoch 7/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.7580 - loss: 1.0903 - val_accuracy: 0.8078 - val_loss: 0.9994
Epoch 8/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - accuracy: 0.8002 - loss: 0

In [16]:
correct_predictions = 0
total_predictions = len(test_strings)

for i, test_string in enumerate(test_strings):
    predicted_output = predict_reverse(test_string)
    is_correct = predicted_output == expected_outputs[i]
    if is_correct:
        correct_predictions += 1
    print(f"input : {test_string}")
    print(f"est. output: {predicted_output}")
    print(f"exp. output: {expected_outputs[i]}")
    print(f"correct: {'맞음' if is_correct else '틀림'}")

accuracy = correct_predictions / total_predictions
print(f"total acc: {accuracy *100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step
input : NtGWuiHtGWxqDIP
est. output: PIDqxWGtHiuWGtN
exp. output: PIDqxWGtHiuWGtN
correct: 맞음
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
input : oPqRYzcqWXHJcRO
est. output: ORcJHXWqczYRqPo
exp. output: ORcJHXWqczYRqPo
correct: 맞음
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
input : dRTmtqFotGDMPNh
est. output: hNPMDGtoFqtmTRd
exp. output: hNPMDGtoFqtmTRd
correct: 맞음
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
input : APAcOBksHtNyVDi
est. output: iDVyNtHskBOcAPA
exp. output: iDVyNtHskBOcAPA
correct: 맞음
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
input : IeWsunPIEysPoAz
est. output: zAoPsyEIPnusWeI
exp. output: zAoPsyEIPnusWeI
correct: 맞음
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
input : YCuXVKgFDquHyvr
est. output: rvyHuqDFgKVXuCY
exp. output: rvyHuqDFgKVXuCY
correct: 맞음
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
input : ajmPoBVTjXDUINA
est. output: ANIUDXjTVBoPmja
exp. output: ANIUDXjTVBoPmja
correct: 맞음
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
input : TjpsBHHNejfQijQ
est. output: